# Load config

In [6]:
from config import config
config = config.Configuration()

In [23]:
# run previous notebooks if necessary
if not "run_prev_notebooks" in locals(): # use case: this notebook
    run_prev_notebooks = False
    %run "1_data_preparation.ipynb"
    %run "2_model.ipynb"

In [12]:
import os
import time
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
timestr = time.strftime("%Y%m%d-%H%M")

# Define callbacks for the early stopping of training, LearningRateScheduler and model checkpointing
checkpoint = ModelCheckpoint(config.model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea; It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience,reduce by a factor of 0.33, new_lr = lr * factor).
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced.
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16)

#early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=15)

log_dir = os.path.join(f'./logs/UNet_{timestr}_{OPTIMIZER_NAME}_{LOSS_NAME}_{config.input_shape[0]}')
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta

In [21]:
%%time
if 1:
    for e in range(config.NB_EPOCHS):
        print('Epoch', e)
        batches = 0
        for x_batch, y_batch in datagen.flow(x_data_expanded, y_data_expanded,
                                            batch_size=config.BATCH_SIZE, subset='training'):
            model.fit(x_batch, y_batch)
            batches += 1
            if batches >= len(x_data_expanded[0]) / config.BATCH_SIZE:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

    print("Manual Training Completed!")

Epoch 0


ValueError: Training and validation subsets have different number of classes after the split. If your numpy arrays are sorted by the label, you might want to shuffle them.

In [31]:
%%time
if 1:
        loss_history = model.fit(expand_dims(train_ds, 0),
                                epochs=config.NB_EPOCHS,
                                steps_per_epoch=len(X_train) // config.BATCH_SIZE, #config.MAX_TRAIN_STEPS, # steps_per_epoch * epochs <= # data instances = 94
                                validation_data=expand_dims(val_ds, 0),
                                # datagen.flow(X_val_np, y_val_np, batch_size=config.VAL_BATCH_SIZE, subset='validation'),
                                validation_steps=len(X_train[0]) // config.BATCH_SIZE,
                                batch_size=config.BATCH_SIZE,
                                callbacks=callbacks_list
                                # use_multiprocessing=True # the generator is not very thread safe
                        )
        print("Training completed.")

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type PrefetchDataset).

In [20]:
%%time
if 1:
        loss_history = model.fit(datagen.flow(x_data_expanded, y_data_expanded,
                                        batch_size=config.BATCH_SIZE, subset='training'),
                                epochs=config.NB_EPOCHS,
                                steps_per_epoch=len(x_data_expanded[0]) // config.BATCH_SIZE, #config.MAX_TRAIN_STEPS, # steps_per_epoch * epochs <= # data instances = 94
                                validation_data=datagen.flow(x_data_expanded, y_data_expanded,
                                        batch_size=8, subset='validation'),
                                # datagen.flow(X_val_np, y_val_np, batch_size=config.VAL_BATCH_SIZE, subset='validation'),
                                validation_steps=len(x_data_expanded[0]) // config.BATCH_SIZE,
                                batch_size=config.BATCH_SIZE,
                                callbacks=callbacks_list
                                # use_multiprocessing=True # the generator is not very thread safe
                        )
        print("Training completed.")

ValueError: Training and validation subsets have different number of classes after the split. If your numpy arrays are sorted by the label, you might want to shuffle them.

In [ ]:
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)

acc     = round(100*loss_history.history['accuracy'][-1], 4)
val_acc = round(100*loss_history.history['val_accuracy'][-1], 4)
timestr = time.strftime("%Y%m%d-%H%M")
model_path = os.path.join(f'{config.model_path}trees_{timestr}_{OPTIMIZER_NAME}_{LOSS_NAME}_{config.input_shape[0]}.h5')
print(model_path)
model.save(model_path)

# The weights without the model architecture can also be saved. Just saving the weights is more efficent.
# weight_path="./saved_weights/UNet/{}/".format(timestr)
# if not os.path.exists(weight_path):
#     os.makedirs(weight_path)
# weight_path=weight_path + "{}_weights.best.hdf5".format('UNet_model')
# print(weight_path)

/home/jovyan/work/saved_data/south_africa_tree_stock/saved_models/UNet_20220314-2341_AdaDelta_weightmap_tversky_256_10.4729_0.0.h5
